In [1]:
!pip install torchinfo

In [2]:
import os
os.chdir("/content/drive/MyDrive/RiverSemanticSegmentation/")
dataset_dir = os.path.normpath("/content/drive/MyDrive/SemanticSegmentationV2/dataset/")

In [3]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from dataloader import Dataset
from torchinfo import summary
import time
import copy

from tqdm import tqdm

from models.unet import UNet
from models.simple import Simple

from dataloader import Dataset

In [4]:
PARAMS = {
    "input_size": 416,
    "output_size": 208,
    "model": "vgg_unet_ks",
    "learning_rate": 0.001,
    "batch_size": 16,
    'epochs': 1000,
    'patience': 20,
    "train_dataset_size": -1,
    "test_dataset_size": -1,
}

In [5]:
!pip install neptune-client
import neptune

neptune.init(project_qualified_name='radek/pth1',
             api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiYmY4YjQ3YjEtNmY5My00MDc2LWI4NzAtMWE5MmUwZjQ1NDE2In0=',
             )

# Create experiment
neptune.create_experiment(params=PARAMS)


     |████████████████████████████████| 112kB 13.5MB/s 
     |████████████████████████████████| 829kB 13.6MB/s 
     |████████████████████████████████| 204kB 28.9MB/s 
     |████████████████████████████████| 163kB 34.1MB/s 
     |████████████████████████████████| 133kB 32.8MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
     |████████████████████████████████| 71kB 9.9MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.5.0-py2.py3-none-any.whl size=166456 sha256=b9a205a5e8b18a6d8f61bf2cb20ac045301b42e204672fed14341d6d096034bb
  Stored in directory: /root/.cache/pip/wheels/80/ba/40/37818997c4e49d0281da4357c7dcb9ecc9c9e01fe95ec2df14
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=22292c1771160d0ff544136d9e79f76e9a7b8b6e7800824948a0d4ca6fe46cae
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for strict-rfc3339: filename=strict_rfc3339-0.7-cp36-none-

Experiment(PTH1-35)

In [6]:
x_train_dir = os.path.join(dataset_dir,"x_train")
y_train_dir = os.path.join(dataset_dir,"y_train")
x_test_dir = os.path.join(dataset_dir,"x_test")
y_test_dir = os.path.join(dataset_dir,"y_test")

In [7]:
train_set = Dataset(x_train_dir, y_train_dir, PARAMS['input_size'], PARAMS['output_size'], PARAMS["train_dataset_size"])
test_set = Dataset(x_test_dir, y_test_dir, PARAMS['input_size'], PARAMS['output_size'], PARAMS["test_dataset_size"])
batch_size = PARAMS['batch_size']
dataloaders = {
    'train': DataLoader(train_set, batch_size=PARAMS['batch_size'], shuffle=True, num_workers=0),
    'val': DataLoader(test_set, batch_size=PARAMS['batch_size'], shuffle=True, num_workers=0)
}


In [8]:
if PARAMS['model'] == "simple":
  from models.simple import Simple
  model = Simple()
elif PARAMS['model'] == "vgg_unet":
  from models.vgg_unet import VggUnet
  model = VggUnet()
elif PARAMS['model'] == "vgg_unet_ks":
  from models.vgg_unet_ks import VggUnetKs
  model = VggUnetKs()
elif PARAMS['model'] == "unet":
  from models.unet import UNet
  model = UNet()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
summary(model, input_size=(PARAMS['batch_size'], 3, PARAMS['input_size'], PARAMS['input_size']))


Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [16, 64, 416, 416]        --
|    └─Conv2d: 2-1                       [16, 64, 416, 416]        1,792
|    └─ReLU: 2-2                         [16, 64, 416, 416]        --
|    └─Conv2d: 2-3                       [16, 64, 416, 416]        36,928
|    └─ReLU: 2-4                         [16, 64, 416, 416]        --
├─MaxPool2d: 1-2                         [16, 64, 208, 208]        --
├─Sequential: 1-3                        [16, 128, 208, 208]       --
|    └─Conv2d: 2-5                       [16, 128, 208, 208]       73,856
|    └─ReLU: 2-6                         [16, 128, 208, 208]       --
|    └─Conv2d: 2-7                       [16, 128, 208, 208]       147,584
|    └─ReLU: 2-8                         [16, 128, 208, 208]       --
├─MaxPool2d: 1-4                         [16, 128, 104, 104]       --
├─Sequential: 1-5                        [16, 256, 104, 104]       --

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [16, 64, 416, 416]        --
|    └─Conv2d: 2-1                       [16, 64, 416, 416]        1,792
|    └─ReLU: 2-2                         [16, 64, 416, 416]        --
|    └─Conv2d: 2-3                       [16, 64, 416, 416]        36,928
|    └─ReLU: 2-4                         [16, 64, 416, 416]        --
├─MaxPool2d: 1-2                         [16, 64, 208, 208]        --
├─Sequential: 1-3                        [16, 128, 208, 208]       --
|    └─Conv2d: 2-5                       [16, 128, 208, 208]       73,856
|    └─ReLU: 2-6                         [16, 128, 208, 208]       --
|    └─Conv2d: 2-7                       [16, 128, 208, 208]       147,584
|    └─ReLU: 2-8                         [16, 128, 208, 208]       --
├─MaxPool2d: 1-4                         [16, 128, 104, 104]       --
├─Sequential: 1-5                        [16, 256, 104, 104]       --

In [9]:
from collections import defaultdict
import torch.nn.functional as F
SMOOTH = 1e-6
def iou_metric(outputs: torch.Tensor, labels: torch.Tensor):
    #print(outputs.min())
    #print(labels.min())
    outputs = outputs.squeeze(1)  # BATCH x 1 x H x W => BATCH x H x W
    labels = labels.squeeze(1)
    intersection = (outputs * labels).sum(2).sum(1)  # Will be zero if Truth=0 or Prediction=0
    union = (outputs + labels).sum(2).sum(1) - intersection  # Will be zzero if both are 0
    iou = (intersection + SMOOTH) / (union + SMOOTH)  # We smooth our devision to avoid 0/0
    return iou.mean()
    

def dice_loss(pred, target, smooth = 1.):
    pred = pred.contiguous()
    target = target.contiguous()    
    intersection = (pred * target).sum(dim=2).sum(dim=2)
    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))
    return loss.mean()

def calc_loss(pred, target, metrics, bce_weight=0.5):
    bce = F.binary_cross_entropy_with_logits(pred, target)
    pred = torch.sigmoid(pred)
    pred = torch.round(pred)
    dice = dice_loss(pred, target)
    loss = bce# * bce_weight + dice * (1 - bce_weight)
    iou = iou_metric(pred, target)
    metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
    metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)
    metrics['iou'] += iou.data.cpu().numpy() * target.size(0)
    return loss

def print_metrics(metrics, epoch_samples, phase):   
    print(epoch_samples) 
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))
        neptune.log_metric(phase+"_"+k, metrics[k] / epoch_samples) #log
    print("{}: {}".format(phase, ", ".join(outputs)))    



In [10]:
#from metrics import *

def train_model(model, dataloaders, optimizer, device, num_epochs=25, patience=-1):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_loss = 1e10
    no_improvement = 0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        since = time.time()

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                for param_group in optimizer.param_groups:
                    print("LR", param_group['lr'])
                    
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            metrics = defaultdict(float)
            epoch_samples = 0
            
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)             

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = calc_loss(outputs, labels, metrics)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                epoch_samples += inputs.size(0)

            print_metrics(metrics, epoch_samples, phase)
            epoch_loss = metrics['loss'] / epoch_samples

            # deep copy the model
            if phase == 'val':
              if epoch_loss < best_loss:
                no_improvement = 0
                print("Accuracy improvement achieved - saving best model.")
                best_loss = epoch_loss
                best_model_wts = copy.deepcopy(model.state_dict())
              else:
                no_improvement += 1
                print("No accuracy improvement since {}/{} epochs.".format(no_improvement,patience))
        time_elapsed = time.time() - since
        print('{:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        if patience >= 0 and no_improvement > patience:
          break
    print('Best val loss: {:4f}'.format(best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [11]:
optimizer_ft = optim.Adam(model.parameters(), lr=PARAMS['learning_rate'])
model = train_model(model, dataloaders, optimizer_ft, device, num_epochs=PARAMS['epochs'], patience=PARAMS['patience'])

  0%|          | 0/151 [00:00<?, ?it/s]

Epoch 0/999
----------
LR 0.001


100%|██████████| 151/151 [47:56<00:00, 19.05s/it]


2412


  0%|          | 0/36 [00:00<?, ?it/s]

...

570
val: bce: 218.464447, dice: 0.174497, loss: 218.464447, iou: 0.736983
No accuracy improvement since 15/15 epochs.
1m 26s
Epoch 33/999
----------
LR 0.001


  0%|          | 0/36 [00:00<?, ?it/s]

2412
train: bce: 0.043786, dice: 0.156216, loss: 0.043786, iou: 0.763059


100%|██████████| 36/36 [00:16<00:00,  2.23it/s]

570
val: bce: 512.751692, dice: 0.199634, loss: 512.751692, iou: 0.700475
No accuracy improvement since 16/15 epochs.
1m 27s
Best val loss: 0.086512


In [12]:
neptune.stop()

In [13]:
torch.save(model.state_dict(),"state_dict.pth")

In [14]:
model.load_state_dict(torch.load("state_dict.pth", map_location="cpu"))

<All keys matched successfully>

In [15]:
from torchvision import transforms
inv_normalize = transforms.Normalize(
   mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
   std=[1/0.229, 1/0.224, 1/0.225]
)

def reverse_transform(inp):
    print(inp.shape)
    inp = inv_normalize(inp)
    inp = inp.numpy()
    inp = np.swapaxes(inp, 1, 3)
    inp = np.swapaxes(inp, 1, 2)
    inp = np.clip(inp, 0, 1)
    inp = (inp * 255).astype(np.uint8)
    
    return inp

In [16]:
import matplotlib.pyplot as plt
def plot_side_by_side(rgb,ground_truth,predict):
  assert rgb.shape[0] == ground_truth.shape[0] == predict.shape[0]
  batch_size = rgb.shape[0]
  fig, axs = plt.subplots(batch_size, 3, figsize=(30,50))
  for i in range(batch_size):
    print(ground_truth[i].min())
    print(predict[i].min())
    axs[i, 0].imshow(rgb[i])
    axs[i, 1].imshow(ground_truth[i])
    axs[i, 2].imshow(predict[i])

In [17]:
# visualize example segmentation

import math
#import helper
model.eval()   # Set model to evaluate mode
test_dataset = Dataset(x_test_dir, y_test_dir, PARAMS['input_size'], PARAMS['output_size'])
test_loader = DataLoader(test_dataset, batch_size=5, shuffle=True, num_workers=0)
inputs, labels = next(iter(test_loader))
inputs = inputs.to(device)
labels = labels.to(device)

labels = labels.data.cpu().numpy()
pred = model(inputs)
pred = torch.sigmoid(pred)
pred = torch.round(pred)
pred = pred.data.cpu().numpy()
inputs = inputs.data.cpu()
#print(inputs.shape)
#print(labels)
#print(torch.sigmoid(torch.from_numpy(pred)).round())

# Change channel-order and make 3 channels for matplot
input_images_rgb = reverse_transform(inputs)#[reverse_transform(x) for x in inputs.cpu()]
#print(input_images_rgb.shape)
# Map each channel (i.e. class) to each color
target_masks_rgb = np.squeeze(labels)#[helper.masks_to_colorimg(x) for x in labels.cpu().numpy()]
#print(target_masks_rgb.shape)
pred_rgb = np.squeeze(pred)#[helper.masks_to_colorimg(x) for x in pred]
#print(input_images_rgb)
plot_side_by_side(input_images_rgb, target_masks_rgb, pred_rgb)

Output hidden; open in https://colab.research.google.com to view.